In [1]:
import pandas as pd
import itertools
import sys
import os
from pathlib import Path
import pickle
import pandas as pd
import json
import openai
import pickle
import dotenv
import random
import time
path = Path(os.getcwd())
sys.path.append(str(path.parent.absolute()))
import propmts
from common_utils import *
from factual_recall_qa.utils import *
from constants import *



In [2]:
setup_directories()
with open('../factual_recall_qa/dataset/icl_query_year_nobel_laureates.pickle', 'rb') as handle:
    df = pickle.load(handle)
with open('../knowledge_validation/dataset/nobel_laureates_attribute_verified.pickle', 'rb') as handle:
    data = pickle.load(handle)
with open('../factual_recall_qa/dataset/icl_query_place_nobel_laureates.pickle', 'rb') as handle:
    df_place = pickle.load(handle)

In [3]:
meta = metadata[NOBEL_LAUREATES]
for concept_class in meta[TARGET_ATTRIBUTES].keys():
    for attribute in meta[TARGET_ATTRIBUTES][concept_class]:
        filtered_df = df[df[TARGET_ATTRIBUTES]==attribute]
        if len(filtered_df)==0:
            continue
        correct  = filtered_df.apply(lambda x: score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
        print(f"{concept_class} {attribute} {len(filtered_df)} Number of correct: {sum(correct)} Number of Hallucinated: {str(len(correct) - sum(correct))}")

year born 500 Number of correct: 460 Number of Hallucinated: 40
year died 499 Number of correct: 4 Number of Hallucinated: 495
year year 500 Number of correct: 498 Number of Hallucinated: 2


In [4]:
meta = metadata[NOBEL_LAUREATES]
for concept_class in meta[TARGET_ATTRIBUTES].keys():
    for attribute in meta[TARGET_ATTRIBUTES][concept_class]:
        filtered_df = df_place[df_place[TARGET_ATTRIBUTES]==attribute]
        if len(filtered_df)==0:
            continue
        correct  = filtered_df.apply(lambda x: score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
        print(f"{concept_class} {attribute} {len(filtered_df)} Number of correct: {sum(correct)} Number of Hallucinated: {str(len(correct) - sum(correct))}")

place borncity 500 Number of correct: 397 Number of Hallucinated: 103
place diedcity 472 Number of correct: 271 Number of Hallucinated: 201
place city 500 Number of correct: 473 Number of Hallucinated: 27


In [5]:
result_entries = df.to_dict('records')
for e in range(len(result_entries)):
    string = result_entries[e][ICL_PROMPT_COLUMN]
    record_found = None
    k =0
    for c in data:
        suffix =[ 
            c[CONTEXTUALISING_ATTRIBUTES][Attribute.FIRST_NAME.value],
        ]
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value])
        suffix = ', '.join(suffix) + ': '
        if string.endswith(suffix):
            record_found = c
            break
    if record_found is None:
        print(string)
        print(result_entries[e][ICL_PROMPT_COLUMN])
    result_entries[e][Attribute.BIRTH_DATE_NOBEL.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.BIRTH_DATE_NOBEL.value])
    result_entries[e][Attribute.DEATH_DATE_NOBEL.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.DEATH_DATE_NOBEL.value])
    result_entries[e][Attribute.YEAR.value] = extract_year(record_found[TARGET_ATTRIBUTES][ConceptClass.YEAR.value][Attribute.YEAR.value])
    

In [6]:
result_entries_place = df_place.to_dict('records')
for e in range(len(result_entries_place)):
    string = result_entries_place[e][ICL_PROMPT_COLUMN]
    record_found = None
    k =0
    for c in data:
        suffix =[ 
            c[CONTEXTUALISING_ATTRIBUTES][Attribute.FIRST_NAME.value],
        ]
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.SURNAME.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.MOTIVATION_NOBEL.value])
        if c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value] is not None:
            suffix.append(c[CONTEXTUALISING_ATTRIBUTES][Attribute.CATEGORY_NOBEL.value])
        suffix = ', '.join(suffix) + ': '
        if string.endswith(suffix):
            record_found = c
            break
    if record_found is None:
        print(string)
        print(result_entries[e][ICL_PROMPT_COLUMN])
    result_entries_place[e][Attribute.BIRTH_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.BIRTH_CITY.value]
    result_entries_place[e][Attribute.DEATH_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.DEATH_CITY.value]
    result_entries_place[e][Attribute.WORK_CITY.value] = record_found[TARGET_ATTRIBUTES][ConceptClass.PLACE.value][Attribute.WORK_CITY.value]
    

In [7]:
df = pd.DataFrame(result_entries)
df_place = pd.DataFrame(result_entries_place)

In [8]:
df.to_csv('dataset/icl_query_response_nobel_laureates_year.csv')
df_place.to_csv('dataset/icl_query_response_nobel_laureates_place.csv')

In [9]:
concept_class = ConceptClass.YEAR.value
hallucinated_mask = df.apply(lambda x: not score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
correct = df.apply(lambda x:  score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
df_hallucinated = df[hallucinated_mask]
df_correct = df[correct]

In [10]:
df_hallucinated[TARGET_ATTRIBUTES].value_counts()

died    495
born     40
year      2
Name: target_attributes, dtype: int64

In [11]:
df_correct[TARGET_ATTRIBUTES].value_counts()

year    498
born    460
died      4
Name: target_attributes, dtype: int64

In [12]:
## Analysis of hallucinated death year
t_attr = Attribute.DEATH_DATE_NOBEL.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_Birth_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_DATE_NOBEL.value]), axis=1)
df_attr['equals_Award_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.YEAR.value]), axis=1)

/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/3249828240.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_Birth_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_DATE_NOBEL.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/3249828240.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_Award_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x

In [13]:
print(f"Number of hallucinated death year: {len(df_attr)}")
print(f"Number of hallucinated death year equals birth year: {sum(df_attr['equal_Birth_year'])}")
print(f"Number of hallucinated death year equals award year: {sum(df_attr['equals_Award_year'])}")
print(f"Number of hallucinated death year equals neither birth year and award year: {sum(~df_attr['equal_Birth_year'] & ~df_attr['equals_Award_year'])}")

Number of hallucinated death year: 495
Number of hallucinated death year equals birth year: 0
Number of hallucinated death year equals award year: 493
Number of hallucinated death year equals neither birth year and award year: 2


In [14]:
## Analysis of hallucinated birth year
t_attr = Attribute.BIRTH_DATE_NOBEL.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_Award_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.YEAR.value]), axis=1)
df_attr['equals_Death_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_DATE_NOBEL.value]), axis=1)

/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1071796119.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_Award_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.YEAR.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1071796119.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_Death_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.D

In [15]:
print(f"Number of hallucinated birth year: {len(df_attr)}")
print(f"Number of hallucinated birth year equals death year: {sum(df_attr['equals_Death_year'])}")
print(f"Number of hallucinated birth year equals award year: {sum(df_attr['equal_Award_year'])}")
print(f"Number of hallucinated birth year equals neither death year and award year: {sum(~df_attr['equal_Award_year'] & ~df_attr['equals_Death_year'])}")

Number of hallucinated birth year: 40
Number of hallucinated birth year equals death year: 0
Number of hallucinated birth year equals award year: 33
Number of hallucinated birth year equals neither death year and award year: 7


In [16]:
## Analysis of hallucinated award year
t_attr = Attribute.YEAR.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_Birth_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_DATE_NOBEL.value]), axis=1)
df_attr['equals_Death_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_DATE_NOBEL.value]), axis=1)

/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/350485155.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_Birth_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_DATE_NOBEL.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/350485155.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_Death_year'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[A

In [17]:
print(f"Number of hallucinated award year: {len(df_attr)}")
print(f"Number of hallucinated award year equals death year: {sum(df_attr['equals_Death_year'])}")
print(f"Number of hallucinated award year equals birth year: {sum(df_attr['equal_Birth_year'])}")
print(f"Number of hallucinated award year equals neither death year and birth year: {sum(~df_attr['equal_Birth_year'] & ~df_attr['equals_Death_year'])}")

Number of hallucinated award year: 2
Number of hallucinated award year equals death year: 0
Number of hallucinated award year equals birth year: 0
Number of hallucinated award year equals neither death year and birth year: 2


In [18]:
concept_class = ConceptClass.PLACE.value
hallucinated_mask = df_place.apply(lambda x: not score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
correct = df_place.apply(lambda x:  score_attribute(concept_class, x[GROUND_TRUTH], x[GPT_4_RESPONSE]), axis=1)
df_hallucinated = df_place[hallucinated_mask]
df_correct = df_place[correct]

In [19]:
## Analysis of hallucinated death city
t_attr = Attribute.DEATH_CITY.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_Birth_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_CITY.value]), axis=1)
df_attr['equals_Work_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.WORK_CITY.value]), axis=1)



/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/2469044563.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_Birth_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_CITY.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/2469044563.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_Work_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attrib

In [20]:
print(f"Number of hallucinated death city: {len(df_attr)}")
print(f"Number of hallucinated death city equals birth city: {sum(df_attr['equal_Birth_city'])}")
print(f"Number of hallucinated death city equals work city: {sum(df_attr['equals_Work_city'])}")
print(f"Number of hallucinated death city equals neither birth city and work city: {sum(~df_attr['equal_Birth_city'] & ~df_attr['equals_Work_city'])}")

Number of hallucinated death city: 201
Number of hallucinated death city equals birth city: 70
Number of hallucinated death city equals work city: 79
Number of hallucinated death city equals neither birth city and work city: 58


In [21]:
## Analysis of hallucinated birth city
t_attr = Attribute.BIRTH_CITY.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_death_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_CITY.value]), axis=1)
df_attr['equals_Work_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.WORK_CITY.value]), axis=1)



/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1820394883.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_death_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_CITY.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1820394883.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_Work_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attrib

In [22]:
print(f"Number of hallucinated birth city: {len(df_attr)}")
print(f"Number of hallucinated birth city equals death city: {sum(df_attr['equal_death_city'])}")
print(f"Number of hallucinated birth city equals work city: {sum(df_attr['equals_Work_city'])}")
print(f"Number of hallucinated birth city equals neither death city and work city: {sum(~df_attr['equal_death_city'] & ~df_attr['equals_Work_city'])}")

Number of hallucinated birth city: 103
Number of hallucinated birth city equals death city: 44
Number of hallucinated birth city equals work city: 54
Number of hallucinated birth city equals neither death city and work city: 32


In [24]:
## Analysis of hallucinated work city
t_attr = Attribute.WORK_CITY.value
df_attr = df_hallucinated[df_hallucinated[TARGET_ATTRIBUTES]==t_attr]

df_attr['equal_death_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_CITY.value]), axis=1)
df_attr['equals_birth_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.BIRTH_CITY.value]), axis=1)

/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1491318869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equal_death_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attribute.DEATH_CITY.value]), axis=1)
/var/folders/w_/wwps8sw93kvb_43n4h0303tw0000gn/T/ipykernel_20125/1491318869.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_attr['equals_birth_city'] = df_attr.apply(lambda x: score_attribute(concept_class, x[GPT_4_RESPONSE], x[Attri

In [25]:
print(f"Number of hallucinated work city: {len(df_attr)}")
print(f"Number of hallucinated work city equals death city: {sum(df_attr['equal_death_city'])}")
print(f"Number of hallucinated work city equals birth city: {sum(df_attr['equals_birth_city'])}")
print(f"Number of hallucinated work city equals neither death city and birth city: {sum(~df_attr['equal_death_city'] & ~df_attr['equals_birth_city'])}")

Number of hallucinated work city: 27
Number of hallucinated work city equals death city: 11
Number of hallucinated work city equals birth city: 4
Number of hallucinated work city equals neither death city and birth city: 13
